In [52]:
using DataFrames, CSV
using JLD
using JuMP, Gurobi
using LinearAlgebra, Random, Printf, StatsBase, CategoricalArrays
using Plots, StatsPlots
using Distributions

In [54]:
races = ["Sakhir", "Jeddah", "Melbourne", "Shanghai", "Baku", "Miami", "Imola", "Monaco", "Barcelona", "Montreal", "Spielberg", "Silverstone", "Budapest", "Spa", "Zandvoort", "Monza", "Singapore", "Suzuka", "Lusail", "Austin", "Mexico City", "Sao Paulo", "Las Vegas","Yas Marina"]
races_circuitref = ["bahrain", "jeddah", "albert_park", "shanghai", "baku", "miami", "imola", "monaco", "catalunya", "villeneuve", "red_bull_ring", "silverstone", "hungaroring", "spa", "zandvoort", "monza", "marina_bay", "suzuka", "losail", "americas", "rodriguez", "interlagos", "las_vegas", "yas_marina"];

In [55]:
#import Data
all_df = CSV.read("circuits.csv", DataFrame)
first(all_df,5)

,circuitId,circuitRef,name,location,country
,Int64,String15,String,String31,String15
1,1,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia
2,2,sepang,Sepang International Circuit,Kuala Lumpur,Malaysia
3,3,bahrain,Bahrain International Circuit,Sakhir,Bahrain
4,4,catalunya,Circuit de Barcelona-Catalunya,Montmeló,Spain
5,5,istanbul,Istanbul Park,Istanbul,Turkey


In [56]:
select!(all_df, "circuitRef", "location", "lat", "lng")

#only keep rows for the races that we care about, i.e. 2024 schedule
all_df[in(races_circuitref).(all_df.circuitRef), :]

,circuitRef,location,lat,lng
,String15,String31,Float64,Float64
1,albert_park,Melbourne,-37.8497,144.968
2,bahrain,Sakhir,26.0325,50.5106
3,catalunya,Montmeló,41.57,2.26111
4,monaco,Monte-Carlo,43.7347,7.42056
5,villeneuve,Montreal,45.5,-73.5228
6,silverstone,Silverstone,52.0786,-1.01694
7,hungaroring,Budapest,47.5789,19.2486
8,spa,Spa,50.4372,5.97139
9,monza,Monza,45.6156,9.28111


In [57]:
using Geodesy

# Gives distance between two circuits in km
function dist(c1, c2)
    circuit1 = all_df[c1, :]
    circuit2 = all_df[c2, :]
    lat1 = circuit1[:lat]
    lng1 = circuit1[:lng]
    lat2 = circuit2[:lat]
    lng2 = circuit2[:lng]
    return euclidean_distance(LLA(lat1, lng1, 0), LLA(lat2, lng2, 0)) / 1000
end

dist(1,12)

12386.36377883742

In [64]:
#create a matrix of distances between each circuit
distance_matrix=[dist(i,j) for i in 1:24, j in 1:24]

24×24 Matrix{Float64}:
     0.0    6067.92   10361.5    12339.2    …   7556.33   10100.7    10082.2
  6067.92      0.0     5818.24    9417.92       4922.78    5418.79   12100.1
 10361.5    5818.24       0.0     4611.4        7542.94     447.152  10994.7
 12339.2    9417.92    4611.4        0.0        9252.61    5017.04    9327.96
 11597.7    7749.32    2536.63    2254.5        8118.92    2941.47   10454.3
 12232.2    9109.4     4255.09     486.382  …   8937.29    4658.66    9644.26
 12317.0   11632.9     9204.84    5699.13       9431.36    9461.91    8271.26
 12320.5    9310.67    4630.56     592.452      8888.76    5021.37    9631.6
 12365.8    9443.78    5024.18    1192.61       8671.2     5392.55    9750.33
 12206.6    9011.25    4339.68     991.471      8548.98    4721.25    9940.26
 11956.8    8436.49    3581.74    1497.65   …   8239.14    3965.21   10262.5
 12386.4    9587.92    4801.12     321.243      9469.08    5207.96    9106.87
 12254.6    9133.65    4542.94    1025.19      